In [1]:
import numpy as np
from qiskit import *
from qiskit.providers.fake_provider import FakeManila, FakeKolkata, FakeMelbourne
from mthree.twirling.twirl_circuit import texmex_data
from mthree.utils import final_measurement_mapping
from mthree import M3Mitigation
from mthree.calibrations import Calibration
from mthree.generators import CompleteGenerator, HadamardGenerator, RandomGenerator
import matplotlib.pyplot as plt
from mthree.utils import expval

In [2]:
backend = FakeMelbourne()

In [35]:
def GHZ(N):
    qc = QuantumCircuit(N, N)
    qc.h(0)
    for i in range(1,N):
        qc.cx(0,i)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=2, seed_transpiler=12345)
    return trans_qc

def m3circuit(N):
    qc = QuantumCircuit(N, N)
    qc.x(range(N))
    qc.h(range(N))

    for kk in range(N // 2, 0, -1):
        qc.ch(kk, kk - 1)
    for kk in range(N // 2, N - 1):
        qc.ch(kk, kk + 1)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=2, seed_transpiler=12345)
    return trans_qc

def donothing(N):
    qc = QuantumCircuit(N, N)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=2, seed_transpiler=12345)
    return trans_qc

def values_from_dict(dict):
    values = []
    for key,value in dict.items():
        values.append(value)
    return values

def sort_counts(dict):
    # sort dictionary of counts with the most common occurence in the beginning
    return {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [62]:
transpiled_circuit = m3circuit(8)
#print(final_measurement_mapping(transpiled_circuit))
qubits = values_from_dict(final_measurement_mapping(transpiled_circuit))
#print(qubits)
op = 'ZZZZZZZZ'

In [82]:
cal = Calibration(backend=backend,qubits=qubits)
cal.calibrate_from_backend(shots=2**13)
calib_counts = cal.to_texmex_counts()

texmex_dat = texmex_data(backend=backend, circuit=transpiled_circuit, shots=2**13)
print(expval(texmex_dat,op))

cal.mitigated_expval_std(texmex_dat,qubits,op)

0.13818359375


(0.2775870524767043, 0.022621809527378127)

In [6]:
# def list_to_dict(input_list):
#     result_dict = {}
#     for idx, value in enumerate(input_list):
#         result_dict[idx] = value
#     return result_dict

# from qiskit.result import marginal_counts
# def permute_bitstring(bitstring, shuffle_pattern):
#     """
#     Permute a bit-string based on the given shuffle pattern.

#     Parameters:
#         bitstring (str): The input bit-string to be permuted.
#         shuffle_pattern (list): The shuffle pattern specifying the new order of bits.

#     Returns:
#         str: Permuted bit-string.
#     """
#     permuted_bitstring = ""

#     for index in shuffle_pattern:
#         permuted_bitstring += bitstring[index]

#     return permuted_bitstring 

# def marginalize_counts(counts, qubits, bit_to_physical_mapping):
#     final_measurement_mapping = list_to_dict(qubits)
#     print(final_measurement_mapping)
#     final_mapping_interchanged = {v: k for k, v in final_measurement_mapping.items()}
#     physical_to_bit_mapping = {v: k for k, v in bit_to_physical_mapping.items()}
    
#     indices_to_keep = [physical_to_bit_mapping[qubit] for qubit in qubits]

#     shuffled_marginalized_counts = marginal_counts(result=counts, indices=indices_to_keep)
    
#     indices_to_shuffle = [final_mapping_interchanged[bit_to_physical_mapping[index]] for index in indices_to_keep]
    
#     marginalized_counts = {permute_bitstring(key, indices_to_shuffle): val for key, val in shuffled_marginalized_counts.items()}
    
#     return marginalized_counts

# def rel_variance_from_expval(x, shots):
#     """
#     Relative Variance of a quantity estimated by summing up {-1,+1} for each shot based on the 
#     observed bistring 

#     Parameters:
#         x (float): Input Expectation value 
#         shots (int): Total Number of shots

#     Returns:
#         float: variance 
#     """
#     return (1-x**2)/(shots * x**2)

# from mthree.utils import expval

# def mitig_expval_std(counts, qubits, calibration_counts, bit_to_physical_mapping, operator=None):
#     """
#     Use the counts data to compute mitigated expectation value of an operator for a given circuit 

#     Parameters:
#         counts (dict): dictionary of counts
#         qubits (list): list of qubits measured at the end of the circuit
#         calibration_counts (dict): dictionary of counts
#         bit_to_physical_mapping (dict): dictionary containing mapping of measured qubits for calibration
#         operator (str or dict or list): String or dict representation of diagonal 
#                                 qubit operators used in computing the expectation
#                                 value.

#     Returns:
#         list: list of results for the mitigated expectation value and uncertainity estimate
#     """
#     # This is needed because counts is a Counts object in Qiskit not a dict.
#     counts = dict(counts)
#     calibration_counts = dict(calibration_counts)

#     # Find number of shots for circuit and calibration
#     shots = sum(counts.values())
#     calib_shots = sum(calibration_counts.values())

#     # marginalize calibration data 
#     marginalized_calibration_counts = marginalize_counts(calibration_counts, qubits, bit_to_physical_mapping)

#     # find the expectation value from the circuit data and the associated uncertainity
#     expvalue  = expval(items=counts, exp_ops=operator)
#     expvalue_rel_variance = rel_variance_from_expval(expvalue, shots)

#     # find the expectation value from the calibration data and the associated uncertainity
#     calib_expval  = expval(items=marginalized_calibration_counts, exp_ops=operator)
#     calib_rel_variance = rel_variance_from_expval(calib_expval, calib_shots)

#     # divide by the calibration expectation value to obtain mitigated expectation value 
#     mitigated_expval = expvalue/calib_expval
#     mitigated_rel_variance = expvalue_rel_variance + calib_rel_variance
#     mitigated_std = mitigated_expval*np.sqrt(mitigated_rel_variance)

#     return (mitigated_expval, mitigated_std)

In [72]:
mit = M3Mitigation(backend)
mit.cals_from_system(qubits=qubits,shots=2**13)

raw = backend.run(transpiled_circuit, shots=2**13).result().get_counts()
print(expval(raw,op))

quasi = mit.apply_correction(raw, qubits, return_mitigation_overhead=True)
quasi.expval_and_stddev(op)

0.08251953125


(0.2829291496290417, 0.032923630802916985)